In [ ]:
import sys
import os
sys.path.append('../..')
#sys.path.append(os.path.join(os.path.dirname(os.path.dirname(__file__))))

import pickle
import shutil
import yaml
import gdist
from lib.rigidpose.sixd_toolkit.pysixd import inout
from lib.utils import listdir_nohidden
from scipy.spatial.distance import cdist
import numpy as np
import png
from PIL import Image

In [ ]:
SIXD_PATH = '/home/lucas/datasets/pose-data/sixd/occluded-linemod-augmented2cc_gdists'

In [ ]:
models_info = inout.load_yaml(os.path.join(SIXD_PATH, 'models', 'models_info.yml'))
models = {}
for obj_id in models_info:
    models[obj_id] = inout.load_ply(os.path.join(SIXD_PATH, 'models', 'obj_{:02}.ply'.format(obj_id)))
    print("Obj {}: {} vertices, {} faces.".format(obj_id, len(models[obj_id]['pts']), len(models[obj_id]['faces'])))

**NOTE:**
- Using the SIXD model instead, since the other one is very rough, has 0.5M faces, disconnected components, and for some reason makes gdist library crash. SIXD model rotated into same coord system, hopefully without issues.
- They do however look quite different... Would be better to re-render with models from SIXD.

In [ ]:
# models[10] = inout.load_ply('/home/lucas/datasets/pose-data/sixd/bop-unzipped/hinterstoisser/models/obj_12.ply')
#models[10] = inout.load_ply('/home/lucas/datasets/pose-data/ply-models-ascii/010_smooth_from_sixd_rotated.ply')

In [ ]:
def find_closest_vtx(x, y, z, vertices):
    assert vertices.shape[1] == 3
    distances = np.linalg.norm(vertices - np.array([[x, y, z]]), axis=1)
    vtx_idx = np.argmin(distances)
    return vtx_idx

In [ ]:
def compute_gdists_on_models(models, models_info):
    gdists = {}
    obj_cnt = 0
    for obj_id, model in models.items():
        obj_cnt += 1
#         if obj_id != 10:
#             continue
        nbr_vtx = model['pts'].shape[0]
        nbr_kp = len(models_info[obj_id]['kp_x'])
        gdists[obj_id] = {}
        for kp_idx, kp_coords in enumerate(zip(models_info[obj_id]['kp_x'], models_info[obj_id]['kp_y'], models_info[obj_id]['kp_z'])):
#             if kp_idx != 10:
#                 continue
            kp_vtx_idx = find_closest_vtx(*kp_coords, model['pts'])
            print("Obj {}/{}: {}, keypoint {}/{}".format(obj_cnt, len(models), obj_id, kp_idx+1, nbr_kp))
#             continue
            gdists[obj_id][kp_idx] = gdist.compute_gdist(
                model['pts'].astype(np.float64),
                model['faces'].astype(np.int32),
                source_indices = np.array([kp_vtx_idx], np.int32),
                #target_indices = np.array(list(range(nbr_vtx)), np.int32),
                #max_distance = 100.0,
            )
    #        colors = gdist_to_kp_per_vtx[:,np.newaxis]
    #        colors = 255.999*(1.0-colors/np.max(colors))
    #        models[obj_id]['colors'][:,:] = colors.astype('uint8')
    #        inout.save_ply(
    #            '/tmp/test.ply',
    #            models[obj_id]['pts'],
    #            pts_colors = models[obj_id]['colors'],
    #            pts_normals = models[obj_id]['normals'],
    #            faces = models[obj_id]['faces'],
    #        )
    #        break
    #    break
    return gdists
gdists = compute_gdists_on_models(models, models_info)

In [ ]:
with open(os.path.join(SIXD_PATH, 'models', 'gdists.yml'), 'w') as f:
    yaml.dump(gdists, f, Dumper=yaml.CDumper)

## Annotate gdists & normals per pixel

TODO
* Read keypoints. Find closest vertex?
* For each object, for each vertex, compute and store geodesic distances to all keypoints
* Loop through all corr maps. For each pixel, lookup and store gdist. Find closest vertex / vertices.
* Store gdists & normal on each pixel

In [ ]:
# Use glob to find all segs. Determine corr paths as well & read them both.
# Define paths to gdist and normal maps
# Use seg & corrs to find closest vertex, and its gdist & normal value.

In [ ]:
def project_to_surface(self, obj_id):
    distances = np.linalg.norm(self.models[obj_id]['pts'] - keypoint[np.newaxis,:], axis=1)
    closest_vtx_idx = np.argmin(distances)
    # Overwrite keypoints with closest vertices:
    return self.models[obj_id]['pts'][closest_vtx_idx_list,:]

In [ ]:
def read_yaml(path):
    with open(path, 'r') as f:
        return yaml.load(f, Loader=yaml.CLoader)

In [ ]:
def read_png(filename, dtype=None, nbr_channels=3):
    with open(filename, 'rb') as f:
        data = png.Reader(f).read()[2]
        if dtype is not None:
            img = np.vstack(map(dtype, data))
        else:
            img = np.vstack(data)
    shape = img.shape
    assert shape[1] % nbr_channels == 0
    img = np.reshape(img, (shape[0], shape[1]//nbr_channels, nbr_channels))
    return img

In [ ]:
def write_png(filename, dtype=None, nbr_channels=3):
    with open(filename, 'wb') as f:
        data = png.Writer(f).read()[2]
        if dtype is not None:
            img = np.vstack(map(dtype, data))
        else:
            img = np.vstack(data)
    shape = img.shape
    assert shape[1] % nbr_channels == 0
    img = np.reshape(img, (shape[0], shape[1]//nbr_channels, nbr_channels))
    return img

In [ ]:
SUBSETS = [subset for subset in listdir_nohidden(SIXD_PATH) if subset.startswith('train') or subset.startswith('test')]

In [ ]:
for subset in SUBSETS:
    if subset not in [
        'train_unoccl',
        #'train_occl',
        #'test_occl',
    ]:
        continue
    seqs = listdir_nohidden(os.path.join(SIXD_PATH, subset))
    #if subset == 'train_unoccl':
    #    seqs = ['driller']
    #elif subset == 'train_occl':
    #    seqs = ['ape']
    #elif subset == 'test_occl':
    #    seqs = ['benchviseblue']
    for seq in seqs:
        rgb_dir = os.path.join(SIXD_PATH, subset, seq, 'rgb')
        instance_seg_dir = os.path.join(SIXD_PATH, subset, seq, 'instance_seg')
        corr_dir = os.path.join(SIXD_PATH, subset, seq, 'obj')
        #normals_dir = os.path.join(SIXD_PATH, subset, seq, 'normals')
        vtx_idx_dir = os.path.join(SIXD_PATH, subset, seq, 'vtx_idx')

        if os.path.exists(vtx_idx_dir):
            shutil.rmtree(vtx_idx_dir)
        os.makedirs(vtx_idx_dir)

        gts = read_yaml(os.path.join(SIXD_PATH, subset, seq, 'gt.yml'))

        fnames = list(sorted(listdir_nohidden(rgb_dir)))
        for j, fname in enumerate(fnames):
            img_idx = int(fname.split('.')[0])

            if (j+1) % 10 == 0:
                print("subset {}, seq {}, frame {}/{}".format(subset, seq, j+1, len(fnames)))

            instance_seg_path = os.path.join(instance_seg_dir, fname)
            corr_path = os.path.join(corr_dir, fname)
            #normals_path = os.path.join(normals_dir, fname)
            vtx_idx_path = os.path.join(vtx_idx_dir, fname)

            # Read segmentation & correspondence map
            corr_map = read_png(corr_path, dtype=np.int16, nbr_channels=3).astype('float64') + 0.5
            instance_seg = np.array(Image.open(instance_seg_path))

            img_height, img_width = instance_seg.shape

            # Vertex index map
            vtx_idx_map = np.zeros((img_height, img_width), dtype='uint32')

            instance_idx = 0
            for gt in gts[img_idx]:
                instance_idx += 1

                mask = instance_seg == instance_idx
                surface_pts = corr_map[mask,:]

                obj_id = gt['obj_id']
                nbr_kp = len(models_info[obj_id]['kp_x'])

                # Lookup closest vertices to surface points
                distance_matrix = cdist(surface_pts, models[obj_id]['pts'], metric='euclidean')
                vtx_idx_map[mask] = np.argmin(distance_matrix, axis=1)

            #assert vtx_idx_map.max() < 2**16
            #Image.fromarray(vtx_idx_map.astype(np.uint16)).save(vtx_idx_path)
            Image.fromarray(vtx_idx_map.astype(np.uint32)).save(vtx_idx_path)